# 信息的定量描述
直观理解： 
若消息发生的概率很大，受信者事先已经有所估计，则该消息的信息量就很小。 
若消息发生的概率很小，受信者感觉到很突然，该消息所含有的信息量就很大。 
所以信息量和概率联系在了一起，信息量可以表示为概率的函数。那么怎样的函数可以用来描述信息量呢？函数f(p)f(p)应该满足以下条件： 
1. f(p)应该是概率p的严格单调递减函数， 
2. 当p=1时，f(p)=0 
3. 当p=0时，f(p)=∞
4. 两个独立事件的联合信息量应该等于它们信息量之和。 
以下是f(p)=−log(p)f(p)=−log(p)的图像，满足以上的所有的要求。
--------------------- 
作者：_席达_ 
来源：CSDN 
原文：https://blog.csdn.net/robin_Xu_shuai/article/details/74011205 
版权声明：本文为博主原创文章，转载请附上博文链接！


# ID3算法熵计算公式

1. 选择分类\\(x_i\\)的概率

2. 信息期望值

自信息和熵的定义
若一个消息xx出现的概率为p，那么这个消息所含有的信息量为 
$$I=−log(p)$$

上式称为消息x的自信息，自信息有两种含义： 
1. 当该消息发生之前，表示发生该消息的不确定性， 
2. 当该消息发生之后，表示消息所含有的信息量。 
信源含有的信息量是信源发出的所有可能消息的平均不确定性，香农把信源所含有的信息量称为熵，是指每个符号所含有的信息量（自信息）的统计平均。若X是一个离散随机变量，概率分布为\\(p(x)=P(X=x)\\),\\(x∈X\\)，那么XX的熵为 
$$H(X)=\sum_{i}^{N}p(xi)I(xi)=−\sum_{i}^{N}p(xi)logp(xi)$$


$$H(X)=-\sum_{i=1}^{n}p(x_i)log_2p(x_i)$$

In [11]:
from math import log
def calcShannonEnt(dataSet):
  numEntries = len(dataSet) #数据集长度
  labelCounts = {}
  for featVec in dataSet: #对数据集内所有判别式初始化为0
    currentLabel = featVec[-1] #区每组数据尾部作为最终判别标签
    if currentLabel not in labelCounts.keys():
      labelCounts[currentLabel] = 0
    labelCounts[currentLabel] += 1
  shannonEnt = 0.0 #香农熵
  for key in labelCounts:
    prob = float(labelCounts[key])/numEntries  #计算每个类标签的出现概率
    shannonEnt -= prob * log(prob,2) 
  return shannonEnt

| 不浮出水面是否可以生存 | 是否有蹼 | 鱼   |
| ---------------------- | -------- | ---- |
| yes                    | yes      | yes  |
| yes                    | yes      | yes  |
| yes                    | no       | no   |
| no                     | yes      | no   |
| no                     | no       | no   |


In [12]:
def createDataSet():
  dataSet = [
    [1,1,'yes'],
    [1,1,'yes'],
    [1,0,'no'],
    [0,1,'no'],
    [0,1,'no']
  ]
  labels = ['no surfacing','flippers']
  return dataSet,labels

# 按照给定特征划分数据集

In [22]:
def splitDataSet(dataSet,axis,value):
  retDataSet = []
  for featVec in dataSet:
    if featVec[axis] == value:
      reducedFeatVec = featVec[:axis]
      reducedFeatVec.extend(featVec[axis+1:])
      retDataSet.append(reducedFeatVec)
  return retDataSet

In [23]:
myDat,labels = createDataSet()
print(myDat)
sn = calcShannonEnt(myDat)
print(sn)
x1 = splitDataSet(myDat,0,1)
x2 = splitDataSet(myDat,0,0)
print(x1,'\n',x2)

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]
0.9709505944546686
[[1, 'yes'], [1, 'yes'], [0, 'no']] 
 [[1, 'no'], [1, 'no']]


# 选择最好的数据集划分方式

In [20]:
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain, bestFeature = 0.0,-1
    for i in range(numFeatures):
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)
        newEntropy = 0.0
        for value in uniqueVals :
            subDataSet = splitDataSet(dataSet,i,value)
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)
        infoGain = baseEntropy - newEntropy
        print('infoGain=',infoGain,'bestFeature=',i,baseEntropy,newEntropy)
        if (infoGain > bestInfoGain):
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature

# 创建树

In [27]:
def createTree(dataSet,labels):
    classList = [example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    if len(dataSet[0]) == 1:
        return majorityCnt(classList)
    bestFeat = chooseBestFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel:{}}
    del(labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet,bestFeat,value),subLabels)
    return myTree
    

In [28]:
def classify(inputTree,featLabels,testVec):
    firstStr = inputTree.keys()[0]
    secondDict = inputTree[firstStr]
    featIndex = featLabels.index(firstStr)
    key = testVec[featIndex]
    valueofFeat = secondDict[key]
    print('+++',firstStr,'xxx',secondDict,'---',key,'>>>',valueofFeat)
    if isinstance(valueofFeat,dict):
        classLabel = classify(valueofFeat,featLabels,testVec)
    else:
        classLabel = valueofFeat
    return  classLabel

In [29]:
myTree = createTree(myDat,labels)

infoGain= 0.4199730940219749 bestFeature= 0 0.9709505944546686 0.5509775004326937
infoGain= 0.17095059445466854 bestFeature= 1 0.9709505944546686 0.8


IndexError: list index out of range